### News

Placeholder

### Installation

In [1]:
#@title Colab Extra Install { display-mode: "form" }
import os
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; _numpy = f'numpy=={numpy.__version__}'; _pil = f'pillow=={PIL.__version__}'
    except: _numpy = "numpy"; _pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    _vllm, _triton = ('vllm==0.9.2', 'triton==3.2.0') if is_t4 else ('vllm==0.15.1', 'triton')
    !uv pip install -qqq --upgrade {_vllm} {_numpy} {_pil} torchvision bitsandbytes xformers unsloth
    !uv pip install -qqq {_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 76.9 MB/s eta 0:00:00:00:0100:01
Using Python 3.12.12 environment at: /usr
Resolved 18 packages in 349ms                                        
Prepared 1 package in 522ms                                              
Uninstalled 1 package in 57ms
Installed 1 package in 42ms                                 
 - transformers==4.57.6
 + transformers==4.56.2
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 2ms                                            
Prepared 1 package in 60ms                                               
Uninstalled 1 package in 1ms
Installed 1 package in 4ms                                  
 - trl==0.24.0
 + trl==0.22.2


### Unsloth

# Project Documentation

## Phase Plans
Phase 1: Environment & Infrastructure Setup

    Hardware & OS Verification: Ensure your local machine is running an NVIDIA RTX 4090 (24GB VRAM) on Linux or Windows/WSL.

    Install Core Dependencies: Set up your Python environment and install the required machine learning and orchestration libraries: unsloth, vllm, fastapi, pyzmq, and pydantic.

    NinjaTrader 8 Preparation: Ensure NinjaTrader 8 is installed with Visual Studio integration enabled for C# development, and import the necessary C# bindings for ZeroMQ (e.g., ZeroMQ.dll and libzmq.dll).

Phase 2: NinjaTrader 8 Data Aggregation (C#)

    Create the Market Officer Script: Write a custom NinjaScript strategy that calculates and aggregates the multi-timeframe data (3m, 15m, and 60m OHLCV) alongside indicators like VWAP and Waddah Attar.

    Implement Apex Risk Metrics: Integrate the trailing drawdown logic into the C# script. Track the CurrentBalance, calculate the highestUnrealizedPeak (which includes Maximum Favorable Excursion), and determine the maxBurnObserved.

    Establish ZeroMQ Publisher: Instantiate a ZeroMQ PUB or PUSH socket within the NT8 script to serialize the market state and account metrics into a lightweight payload and broadcast it at the close of every 3-minute bar.

Phase 3: FastAPI Orchestration Engine (Python)

    Initialize the Async Server: Create a FastAPI application to serve as the central bridge between NinjaTrader and the LLM.

    Bind ZeroMQ to the Event Loop: Hook a ZeroMQ SUB or PULL socket into the FastAPI asyncio event loop so it can continuously listen for the NT8 payload without blocking other processes.

    Prompt Engineering Engine: Write a function within FastAPI that receives the NT8 payload and dynamically injects the data into the "Multi-Timeframe Market Officer" prompt template.

Phase 4: Model Fine-Tuning with Unsloth & GRPO

    Load Qwen 3-8B: Use Unsloth to load the Qwen 3-8B model using 4-bit dynamic quantization. This reduces the memory footprint from ~16GB to roughly 6GB, leaving enough VRAM on your RTX 4090 for the necessary RL context windows.

    Define Custom Reward Functions: Create Python-based reward functions for the GRPOTrainer :

        Format Reward: Uses regex to verify the output contains <think> tags followed by valid JSON.

        Apex Efficiency Reward: Calculates (Total Net Profit) / (Max Burn Observed) to penalize trades with deep intra-trade pullbacks.

        MAE Adherence Penalty: Applies a massive penalty if the model's simulated stop-loss distance exceeds 30% of the start-of-day profit balance.

    Execute RLVR: Run the GRPO training loop on historical multi-timeframe data to align the model's behavior with the strict proprietary trading rules.

Phase 5: vLLM Structured Inference

    Deploy the Inference Engine: Load your fine-tuned Unsloth model into a local vLLM server to handle high-throughput generation.

    Enable Reasoning Parser: Start the vLLM server with the appropriate reasoning parser flag (e.g., --reasoning-parser qwen3) so the engine properly handles the <think> reasoning traces.

    Implement Guided Decoding: Define a rigorous Python Pydantic schema that matches your execution JSON format. Pass this schema to the vLLM API request using the guided_json or response_format parameter. This ensures the tokens generated after the closing </think> tag perfectly adhere to the JSON schema.

Phase 6: The Execution Loop

    Inference Request: FastAPI asynchronously sends the formatted prompt to the local vLLM engine.

    Parse Dual-Stream Output: Once the generation is complete, FastAPI extracts the human-readable reasoning from the <think> tags and the structured JSON object.

    Monitor & Execute:

        Push the reasoning summary to a Discord or Telegram webhook for human oversight.

        Serialize the JSON execution block and send it back to NinjaTrader 8 via a ZeroMQ socket to immediately place the LONG, SHORT, or FLAT order based on the LLM's logic.

## ARA 2.0 PRD
# **Advanced Architectural Paradigms for LLM-Driven Algorithmic Trading: Enhancing the Apex Reasoning Agent via GRPO and Multi-Timeframe Orchestration**

The integration of artificial intelligence into quantitative finance has undergone a fundamental architectural transformation during the 2025–2026 period. The industry has shifted decisively away from utilizing large language models (LLMs) purely for auxiliary tasks—such as sentiment analysis or text summarization—toward the deployment of autonomous, reasoning-first agentic systems capable of direct market execution.1 In prior iterations of algorithmic trading, traditional deep learning models and heuristic-based bots struggled with the non-stationary, highly stochastic nature of financial markets, frequently failing to adapt to sudden regime shifts.2 However, the advent of advanced reasoning models, most notably the DeepSeek-R1 lineage and the Qwen 3 series, has established a new paradigm.4 These models do not merely predict the next token based on statistical frequency; they generate latent cognitive traces, allowing them to evaluate multi-dimensional market data, cross-reference conflicting technical indicators, and synthesize complex logic before committing to a final action.5  
The development of the Apex Reasoning Agent (ARA) Version 2.0 represents the culmination of this research, specifically tailored to navigate the stringent, highly regulated environments of proprietary trading firms like Apex Trader Funding. To successfully pass and manage these evaluations, an automated system must achieve a delicate equilibrium: it must possess the sophisticated multi-timeframe analytical capabilities of an institutional trader, operate within the strict hardware constraints of a local consumer-grade GPU (NVIDIA RTX 4090), and execute orders with sub-millisecond latency. Achieving this requires a tightly integrated stack comprising Qwen 3-8B as the base cognitive engine, Unsloth for memory-efficient reinforcement learning, Group Relative Policy Optimization (GRPO) for behavioral alignment, vLLM for deterministic structured decoding, and a ZeroMQ-to-FastAPI bridge for high-frequency execution.7

## **The Evolution of Multi-Agent and Multi-Timeframe Financial Architectures**

In the context of high-frequency and intraday trading, recent empirical research has demonstrated that monolithic, single-prompt LLM architectures are insufficient for navigating the complexities of modern order flow.11 Frameworks such as QuantAgent and TradingAgents have proven that decomposing financial decision-making into specialized, role-based workflows yields significantly higher predictive accuracy, improved Sharpe ratios, and superior risk-adjusted returns.11  
The QuantAgent framework, for instance, isolates market analysis into distinct modules: an Indicator Agent that computes momentum extremes (e.g., RSI, MACD), a Pattern Agent that identifies structural formations, a Trend Agent that maps consolidation zones, and a Risk Agent that synthesizes these vectors to formulate an actionable directive.14 This decomposition mirrors the operational dynamics of real-world proprietary trading desks.12 The ARA 2.0 architecture adapts this multi-agent philosophy into a "Multi-Timeframe Market Officer" paradigm. Rather than instantiating separate LLMs, which would exceed the memory capacity of a single RTX 4090, the ARA utilizes a highly engineered, single-inference prompt that forces the Qwen 3-8B model to sequentially adopt these specialized perspectives across distinct temporal horizons.12  
The analysis begins at the macro level (60-minute horizon) to establish a structural trend bias, identifying major institutional supply and demand liquidity zones. It then narrows to the intermediate level (15-minute horizon) to evaluate momentum pullbacks relative to the Volume Weighted Average Price (VWAP). Finally, it assesses the micro level (3-minute horizon) to identify immediate order flow imbalances and indicator confluence, such as a Waddah Attar momentum squeeze.11 Furthermore, to filter out false liquidity and algorithmic spoofing, the Market Officer is tasked with performing a cross-asset correlation check, evaluating the structural divergence between the Nasdaq 100 (NQ), S\&P 500 (ES), and the Japanese Yen (6J). This hierarchical reasoning process ensures that execution triggers on the 3-minute chart are only authorized when they are mathematically aligned with the macro-structural environment, drastically reducing the probability of maximum adverse excursion (MAE) breaches.11

## **Hardware Constraints and Base Model Optimization: The RTX 4090 Context**

Deploying a reasoning model locally for real-time, uninterrupted market execution requires navigating severe physical memory constraints. The NVIDIA RTX 4090, featuring the Ada Lovelace microarchitecture, 16,384 CUDA cores, and 24GB of GDDR6X VRAM, provides an exceptional balance of compute capability (82.6 TFLOPS) and memory bandwidth.15 However, the 24GB VRAM ceiling inherently limits the deployment of massive, unquantized parameter models, especially when factoring in the continuous memory overhead required for the KV-cache during extended context ingestion.15  
In offline benchmarking, the RTX 4090 achieves optimal inference throughput and operational stability for models under the 14-billion parameter threshold.15 The Qwen 3-8B model emerges as the mathematically ideal candidate for this specific hardware footprint. As a state-of-the-art causal language model, Qwen 3-8B (comprising 8.2 billion total parameters and 36 layers) natively supports a seamless switching mechanism between a "thinking mode" for complex logical deduction and a "non-thinking mode" for rapid conversational generation.17 It surpasses previous distilled models in mathematics, code generation, and commonsense reasoning, all of which are highly transferable to quantitative financial analysis.18  
To fit this 8B model alongside the necessary KV-cache for continuous multi-timeframe OHLCV data streams, advanced memory optimization frameworks are mandatory. The Unsloth library has established itself as the definitive standard for this layer of the architecture. By utilizing Dynamic 4-bit quantization, Unsloth reduces the memory footprint of an 8B model by approximately 75%—shrinking it from roughly 16GB in standard BF16 precision to a highly manageable \~6GB footprint.20 This drastic compression leaves ample VRAM for context windows and the generation phase of reinforcement learning.20  
Unsloth's proprietary memory management algorithms, particularly its cross-entropy optimizations, Triton kernels, and the introduction of the "Standby" feature, minimize VRAM degradation during training and inference.7 By configuring the gpu\_memory\_utilization to its maximum operational capacity (e.g., 0.95), the Unsloth framework efficiently manages intermediate activations asynchronously, allowing for context lengths up to 8x longer than traditional Flash Attention 2 implementations.7 This specific optimization is the technological linchpin that makes it feasible to continuously feed raw NinjaTrader order flow into a local reasoning model without triggering fatal out-of-memory (OOM) errors during volatile, high-tick market sessions.7

### **Table 1: Hardware Optimization Metrics for the RTX 4090 (24GB) via Unsloth**

| Architectural Metric | Standard HF Implementation (BF16) | Unsloth Dynamic 4-Bit Optimization | Hardware Impact (RTX 4090\) |
| :---- | :---- | :---- | :---- |
| **Model Footprint (Qwen 3-8B)** | \~16.0 GB VRAM | \~6.0 GB VRAM | Frees \>10GB for KV Cache and execution |
| **GRPO Training VRAM Overhead** | \> 42.0 GB (Results in OOM) | \~15.0 GB VRAM | Permits local RLVR training on a single GPU |
| **Context Length Capacity** | \~8,192 Tokens | \~32,768+ Tokens | Enables deep ingestion of 60m/15m/3m datasets |
| **Inference Throughput (Tokens/s)** | Baseline | \~1.5x \- 2.0x Faster | Crucial for sub-second intraday trade execution |

## **The Mathematical Foundation of GRPO for Financial Alignment**

While supervised fine-tuning (SFT) can teach an LLM the syntax of a trading strategy, aligning the model's behavioral policy with strict risk-management rules requires Reinforcement Learning (RL). Traditional alignment has relied heavily on Proximal Policy Optimization (PPO), a method that, while highly effective, is computationally exorbitant. PPO requires maintaining multiple models simultaneously in memory: the Actor (the policy being trained), the Reference model, the Reward model, and the Critic (the Value model).24 The Value model, which estimates the expected future rewards of a given state to calculate the advantage function, mirrors the parameter size of the policy model, effectively doubling the computational overhead and rendering local RTX 4090 training mathematically impossible.25  
Group Relative Policy Optimization (GRPO), pioneered during the development of DeepSeekMath and the R1 reasoning models, revolutionizes this pipeline by completely eliminating the Critic/Value model.8 Instead of relying on a secondary neural network to estimate a baseline, GRPO leverages empirical, group-based advantage estimation.27 For a given input state (e.g., a specific configuration of multi-timeframe market indicators), the policy model generates a group of $N$ distinct outputs (actions/reasoning traces).27 The advantage $A\_i$ for each specific output is calculated by evaluating the raw rewards $r\_i$ across the group, calculating the mean $\\mu$ and standard deviation $\\sigma$, and standardizing the results:

$$A\_i \= \\frac{r\_i \- \\mu}{\\sigma}$$  
This mathematical simplification drastically reduces memory consumption, cuts compute costs by approximately 50%, and inherently stabilizes the training process by grounding the advantage calculation in the empirical distribution of the current policy.26 Furthermore, GRPO natively supports Reinforcement Learning with Verifiable Rewards (RLVR).24 In environments like mathematical theorem proving or algorithmic trading, correctness is not subjective or reliant on human preference; it is objectively verifiable.24 If a model generates a trading output that violates a mathematical constraint (e.g., placing a stop loss that exceeds a risk limit), a programmatic, rule-based python function can assign a definitive scalar penalty without requiring an AI-based Reward model.24

## **Decoding the Apex Trader Funding Constraints for 2026**

To understand why the GRPO reward functions must prioritize drawdown mitigation over raw yield maximization, one must thoroughly analyze the exact regulatory architecture of the Apex Trader Funding Performance Accounts.

### **The Apex Trailing Drawdown Logic (The "Burn" Metric)**

The paramount constraint in an Apex account is the **Maximum Trailing Drawdown** (which we mathematically define as "Burn"). The trailing threshold is calculated based on the *highest live unrealized value* reached during open trades, not just the closed trade value.44 To encode this into the RL environment, the GRPO reward function utilizes the following simulated variables:

1. **highestUnrealizedPeak:** The absolute highest equity value reached during a trade (Current Balance \+ Maximum Favorable Excursion \[MFE\]).44  
2. **currentAccountBalance:** The real-time account balance.44  
3. **maxBurnObserved:** The maximum distance ever recorded between the highestUnrealizedPeak and the currentAccountBalance.44

**Practical Example:** If your starting balance is $50,000 with a $2,500 allowed drawdown, your liquidation threshold begins at $47,500. If you enter a trade and your balance temporarily peaks at $50,875 (highestUnrealizedPeak), but you close the trade at $50,100 (currentAccountBalance), your threshold permanently trails the peak. It is now anchored $2,500 behind the $50,875 mark, meaning your new liquidation threshold is $48,375. You essentially "burned" $775 of your drawdown buffer even on a winning trade.44  
To ensure survival, the LLM agent is trained to optimize the **Apex Efficiency Ratio**, defined as:

$$\\text{Apex Efficiency Ratio} \= \\frac{\\text{Total Net Profit}}{\\text{Max Burn Observed}}$$  
High profitability with deep intra-trade pullbacks destroys this ratio and results in negative GRPO rewards, coercing the model to utilize hyper-tight stops and capture immediate momentum.44  
Equally critical is the **30% Consistency Rule**. This dictates that no single trading day's profit can exceed 30% of the trader's total accumulated net profit at the time of a payout request.30 The mathematical formula governing compliance is absolute: $\\text{Highest Profit Day} \\div 0.3 \= \\text{Minimum Total Profit Required}$.31 Therefore, the LLM agent must be explicitly trained to target consistent, moderate daily base hits rather than outlier windfall trades.30

## **Constrained Decoding and Dual-Stream Output Validation**

One of the most complex engineering challenges in deploying LLMs for deterministic algorithmic trading is the fundamental conflict between the probabilistic, fluid nature of reasoning text and the rigid, syntactic requirements of machine-executable code.32 Traditional applications rely on unpredictable regex parsing to extract JSON from raw strings, leading to catastrophic system failures when the model inevitably hallucinates an extra comma, wraps the output in markdown, includes trailing conversational commentary, or deviates from the requested key structures.33  
The resolution to this architectural flaw is found in the deployment of the vLLM inference server utilizing **Structured Outputs and Guided Decoding**.9 Rather than hoping the model generates valid syntax, guided decoding intervenes directly at the token generation level.34 When the inference engine calculates the logit probabilities for the next possible token, the constrained decoding backend (utilizing highly optimized tools like XGrammar or llguidance) compiles a user-defined Pydantic schema into a Deterministic Finite Automaton (DFA).35 It then applies a dynamic bitmask over the model's vocabulary, forcing the model to select only grammatically valid sequences.34  
By initializing the server with specific reasoning flags (e.g., \--reasoning-parser qwen3), the guided decoding engine temporarily suspends the strict JSON constraint to allow the model to operate probabilistically within the \<think\> tags.37 The moment the state tracker detects the closing \</think\> token, it immediately reactivates the XGrammar mask, forcing all subsequent tokens to perfectly conform to the Pydantic schema.37 This guarantees that human operators receive rich, explainable trade logic, while the downstream trading platform receives flawless JSON commands 100% of the time.

## **Low-Latency Asynchronous Execution: NinjaTrader 8 to FastAPI Integration**

NinjaTrader 8 (NT8), a dominant platform in the proprietary funding space, operates on a monolithic C\# architecture and does not natively interface with Python environments.38 Building a traditional REST API bridge between NT8 and Python introduces unacceptable HTTP protocol overhead and latency.  
The optimal architectural solution requires a raw, low-latency inter-process communication (IPC) protocol using **ZeroMQ**.39 Within the NT8 environment, a custom C\# strategy module is deployed, aggregating the exact state of the market (OHLCV arrays, indicators, and the real-time Apex account balance) and pushing it instantly across a local ZeroMQ PUB or PUSH socket.39  
Simultaneously, an asynchronous **FastAPI** application operates as the central orchestrator, binding a ZeroMQ SUB or PULL socket directly into its event loop.10 Upon receiving the market snapshot, FastAPI dynamically injects it into the prompt template and hands it off asynchronously to the local vLLM engine.42 Upon completion of the Dual-Stream Output, the human-readable summary is pushed to a webhook for trader monitoring, while the structured JSON execution command is instantly pushed back through ZeroMQ to NT8 to submit the order.38

# ---

**PRD: Apex Reasoning Agent (ARA) \- Version 2.0**

**Model Base:** Qwen 3-8B (Thinking Mode Enabled)  
**Hardware Target:** NVIDIA RTX 4090 (24GB VRAM)  
**Architecture:** Hierarchical Multi-Timeframe Reasoning via ZeroMQ/FastAPI

## **1\. Executive Summary**

The ARA is a specialized trading LLM designed to pass and manage Apex Trader Funding accounts. It operates by first conducting a multi-timeframe "Market Officer" analysis, generating a human-readable trading thesis, and finally outputting a structured JSON object containing both execution parameters and market regime metadata for NinjaTrader 8 logging. Crucially, the system is engineered to optimize for the Apex "Trailing Drawdown" logic, prioritizing the mitigation of unrealized intra-trade pullbacks (MFE degradation) over maximum hypothetical yield.44

## **2\. Hierarchical Multi-Timeframe Roles**

### **2.1 The Market Officer (Multi-Timeframe Analyst)**

The Officer is responsible for "Top-Down" analysis. It evaluates data across three specific horizons to determine the current market environment:

* **60-Min Analysis:** Structural trend and major supply/demand zones.  
* **15-Min Analysis:** Intermediate momentum and VWAP positioning.  
* **3-Min Analysis:** Immediate order flow and indicator confluence (Waddah/Squeeze).  
* **Cross-Asset Check:** Correlation/Divergence between NQ, ES, and 6J (Yen).

### **2.2 The Trader (Execution Specialist)**

* **Responsibility:** Converts the Officer's "Bias" into a specific entry.  
* **Constraint:** Must prioritize the **Apex Efficiency Ratio** (Total Net Profit / Max Burn Observed) over raw win rate.44

## **3\. Output Protocol (Dual-Stream Reasoning \+ JSON)**

The model must output its response in two distinct sections, strictly enforced by vLLM Guided Decoding and the \--reasoning-parser qwen3 flag.

### **3.1 Human-Readable Summary (The "Why")**

A concise natural language explanation of the trade, generated probabilistically within the \<think\> tags.

* *Format:* "Because \[Market Officer Logic\], I am taking a \[Action\] at \[Price\] with a stop at to target."

### **3.2 Machine-Readable JSON (The "Metadata")**

Immediately following the reasoning trace, a structured block containing execution commands and the Officer’s regime classifications, constrained by a predefined Pydantic schema.

## **4\. Functional Requirements**

### **4.1 Functional Data Pipeline**

For every 3-minute bar, the prompt sent to Qwen 3-8B must include:

* **NQ Data:** 3m, 15m, and 60m OHLCV \+ Indicators.  
* **ES Data:** 3m and 60m price action (for correlation).  
* **Account Stats:** Current Balance, Current Peak (highestUnrealizedPeak), and Distance to Liquidation.44

### **4.2 Updated Output Schema**

The model's final response must follow this exact template:  
---

---

JSON

{  
  "execution": {  
    "action": "LONG",  
    "entry\_price": 18550.25,  
    "stop\_loss": 18532.50,  
    "take\_profit": 18595.00,  
    "confidence\_score": 0.82  
  },  
  "market\_regime": {  
    "tf\_60min": "Bullish\_Trend",  
    "tf\_15min": "Mean\_Reversion",  
    "tf\_3min": "Consolidation\_Squeeze",  
    "correlation\_state": "NQ\_ES\_Aligned",  
    "volatility\_state": "Expanding"  
  },  
  "risk\_management": {  
    "expected\_burn": 17.75,  
    "apex\_efficiency\_target": 2.5  
  }  
}

## **5\. Training & Reward Logic (GRPO)**

The Qwen 3-8B model is fine-tuned using Unsloth's GRPO framework, integrating Reinforcement Learning with Verifiable Rewards (RLVR) to strictly enforce proprietary trading rules.

### **5.1 The "Regime Accuracy" Reward**

The model receives rewards for **Regime Consistency**:

* **Reward (+1.0):** If the 60min regime is labeled "Bullish" and the 60min Close \> 60min EMA.  
* **Penalty (-1.0):** If the model labels a regime "Trending" while the price is in a 15-minute tight range.

### **5.2 The "Reasoning-Action" Alignment**

* **Penalty (-2.0):** If the Natural Language Summary suggests a "Short" but the JSON action is "LONG."

### **5.3 The Apex Efficiency Ratio Reward (Drawdown Mitigation)**

To prevent the bot from relying on deep stop-losses, the GRPO verifier calculates the expected **Burn** over historical backtest segments.44

* The reward function tracks the highestUnrealizedPeak (the Maximum Favorable Excursion simulated during the trade) and subtracts the currentAccountBalance to find the maxBurnObserved.44  
* **Reward Scalar:** Calculated by the formula (Total Net Profit) / (Max Burn Observed).  
* **Penalty (-5.0):** Applied if the proposed stop\_loss distance mathematically allows the maxBurnObserved to breach the remaining Apex trailing threshold.44

## **6\. Technical Architecture (The Loop)**

1. **Data Collection:** NinjaTrader 8 exports 3m, 15m, and 60m snapshots via ZeroMQ to FastAPI.  
2. **Prompt Engineering:** FastAPI formats the "Multi-Timeframe Context."  
3. **Inference:** Qwen 3-8B (vLLM with Guided Decoding) generates the Summary \+ JSON on the 4090\.  
4. **Parsing:** FastAPI uses a regex/parser to split the Human Summary from the JSON.  
5. **Execution:**  
   * **Human Summary:** Sent to a Telegram/Discord bot for trader monitoring.  
   * **JSON:** Sent back to NinjaTrader 8 via ZeroMQ to place the order and log the "Regime Metadata."

## **7\. Success Criteria**

1. **Zero Syntax Failures:** 100% of outputs must contain a valid, parsable JSON block due to vLLM XGrammar backend.  
2. **Burn Awareness:** The model must show a 30% reduction in maxBurnObserved compared to a standard EMA-crossover bot.44  
3. **Regime Intelligence:** The model must correctly identify "Consolidation\_Squeeze" regimes and default to FLAT during those periods to avoid the Apex "choppy market" trap.

## **8\. Directory of Important Resources**

To facilitate the immediate development, fine-tuning, and deployment of the ARA 2.0 system, developers must rely on the following 2026 frameworks and documentation:  
**Unsloth Optimization & Fine-Tuning:**

* **Unsloth Official Documentation:** Detailed guides for VRAM reduction, model checkpointing, and evaluation. ([https://unsloth.ai/docs](https://unsloth.ai/docs))45  
* **Qwen 3 Fine-Tuning Guide:** Specific configurations for preserving Qwen 3's \<think\> tags and reasoning capabilities using Unsloth. ([https://unsloth.ai/docs/models/qwen3-how-to-run-and-fine-tune](https://unsloth.ai/docs/models/qwen3-how-to-run-and-fine-tune))46  
* **Unsloth Colab Notebooks:** Hosted notebooks for rapid SFT and RLVR testing. ([https://unsloth.ai/docs/get-started/unsloth-notebooks](https://unsloth.ai/docs/get-started/unsloth-notebooks))47

**Reinforcement Learning (GRPO):**

* **Unsloth GRPO / RL Guide:** Comprehensive walkthrough on implementing custom Python-based reward functions (e.g., Apex Efficiency Ratio verifiers) directly into the GRPO pipeline. ([https://unsloth.ai/docs/get-started/reinforcement-learning-rl-guide](https://unsloth.ai/docs/get-started/reinforcement-learning-rl-guide))24

**Multi-Agent Trading Logic:**

* **QuantAgent Research (2026):** Theoretical framework for utilizing multi-timeframe price-driven agents (Indicator, Pattern, Trend, Risk) for high-frequency trading. ([https://arxiv.org/abs/2509.09995](https://arxiv.org/abs/2509.09995))48  
* **TradingAgents Framework:** Architectural guidance on formatting Market Officer roles and synthesizing debate logic. ([https://arxiv.org/abs/2412.20138](https://arxiv.org/abs/2412.20138))49

#### **Works cited**

1. Evaluating AI agents: Real-world lessons from building agentic systems at Amazon \- AWS, accessed February 19, 2026, [https://aws.amazon.com/blogs/machine-learning/evaluating-ai-agents-real-world-lessons-from-building-agentic-systems-at-amazon/](https://aws.amazon.com/blogs/machine-learning/evaluating-ai-agents-real-world-lessons-from-building-agentic-systems-at-amazon/)  
2. Multi-Model Trading Framework Mastery \- PickMyTrade, accessed February 19, 2026, [https://blog.pickmytrade.trade/multi-model-trading-framework-2026-guide/](https://blog.pickmytrade.trade/multi-model-trading-framework-2026-guide/)  
3. ATLAS: Adaptive Trading with LLM AgentS Through Dynamic Prompt Optimization and Multi-Agent Coordination \- arXiv, accessed February 19, 2026, [https://arxiv.org/html/2510.15949v2](https://arxiv.org/html/2510.15949v2)  
4. Top 10 Open-source Reasoning Models in 2026 \- Clarifai, accessed February 19, 2026, [https://www.clarifai.com/blog/top-10-open-source-reasoning-models-in-2026](https://www.clarifai.com/blog/top-10-open-source-reasoning-models-in-2026)  
5. Qwen3: Think Deeper, Act Faster | Qwen, accessed February 19, 2026, [https://qwenlm.github.io/blog/qwen3/](https://qwenlm.github.io/blog/qwen3/)  
6. Key Concepts \- Qwen \- Read the Docs, accessed February 19, 2026, [https://qwen.readthedocs.io/en/latest/getting\_started/concepts.html](https://qwen.readthedocs.io/en/latest/getting_started/concepts.html)  
7. Memory Efficient RL | Unsloth Documentation, accessed February 19, 2026, [https://unsloth.ai/docs/get-started/reinforcement-learning-rl-guide/memory-efficient-rl](https://unsloth.ai/docs/get-started/reinforcement-learning-rl-guide/memory-efficient-rl)  
8. Understanding the Math Behind GRPO — DeepSeek-R1-Zero | by Yugen.ai \- Medium, accessed February 19, 2026, [https://medium.com/yugen-ai-technology-blog/understanding-the-math-behind-grpo-deepseek-r1-zero-9fb15e103a0a](https://medium.com/yugen-ai-technology-blog/understanding-the-math-behind-grpo-deepseek-r1-zero-9fb15e103a0a)  
9. Structured Outputs \- vLLM, accessed February 19, 2026, [https://docs.vllm.ai/en/latest/features/structured\_outputs/](https://docs.vllm.ai/en/latest/features/structured_outputs/)  
10. Architecting Scalable FastAPI Systems for Large Language Model (LLM) Applications and External Integrations | by Ali moradi | Medium, accessed February 19, 2026, [https://medium.com/@moradikor296/architecting-scalable-fastapi-systems-for-large-language-model-llm-applications-and-external-cf72f76ad849](https://medium.com/@moradikor296/architecting-scalable-fastapi-systems-for-large-language-model-llm-applications-and-external-cf72f76ad849)  
11. QuantAgent: Price-Driven Multi-Agent LLMs for High-Frequency Trading | OpenReview, accessed February 19, 2026, [https://openreview.net/forum?id=fdKmhFYcQv](https://openreview.net/forum?id=fdKmhFYcQv)  
12. Your Guide to the TradingAgents Multi-Agent LLM Framework | DigitalOcean, accessed February 19, 2026, [https://www.digitalocean.com/resources/articles/tradingagents-llm-framework](https://www.digitalocean.com/resources/articles/tradingagents-llm-framework)  
13. TradingAgents: Multi-Agents LLM Financial Trading Framework \- GitHub, accessed February 19, 2026, [https://github.com/TauricResearch/TradingAgents](https://github.com/TauricResearch/TradingAgents)  
14. Y-Research-SBU/QuantAgent \- GitHub, accessed February 19, 2026, [https://github.com/Y-Research-SBU/QuantAgent](https://github.com/Y-Research-SBU/QuantAgent)  
15. RTX4090 vLLM Benchmark: Best GPU for LLMs Below 8B on Hugging Face, accessed February 19, 2026, [https://www.databasemart.com/blog/vllm-gpu-benchmark-rtx4090](https://www.databasemart.com/blog/vllm-gpu-benchmark-rtx4090)  
16. 30% Negative P\&L Rule \- Maximum Adverse Excursion (MAE) \- Apex Trader Funding, accessed February 19, 2026, [https://support.apextraderfunding.com/hc/en-us/articles/40463232267035-30-Negative-P-L-Rule-Maximum-Adverse-Excursion-MAE](https://support.apextraderfunding.com/hc/en-us/articles/40463232267035-30-Negative-P-L-Rule-Maximum-Adverse-Excursion-MAE)  
17. unsloth/Qwen3-8B-bnb-4bit \- Hugging Face, accessed February 19, 2026, [https://huggingface.co/unsloth/Qwen3-8B-bnb-4bit](https://huggingface.co/unsloth/Qwen3-8B-bnb-4bit)  
18. Qwen3 Highlights \- NVIDIA Developer, accessed February 19, 2026, [https://developer.nvidia.com/downloads/assets/ace/model\_card/qwen3-8b-instruct.pdf](https://developer.nvidia.com/downloads/assets/ace/model_card/qwen3-8b-instruct.pdf)  
19. Qwen/Qwen3-8B \- Hugging Face, accessed February 19, 2026, [https://huggingface.co/Qwen/Qwen3-8B](https://huggingface.co/Qwen/Qwen3-8B)  
20. Fine-Tuning LLM with Unsloth: A Practical Guide to Training Models like Qwen3 8B on Consumer GPU | by İsmail Kağan Acar | Medium, accessed February 19, 2026, [https://medium.com/@acarismailkagan/fine-tuning-llm-with-unsloth-a-practical-guide-to-training-models-like-qwen3-8b-on-a-consumer-gpu-4116088a207c](https://medium.com/@acarismailkagan/fine-tuning-llm-with-unsloth-a-practical-guide-to-training-models-like-qwen3-8b-on-a-consumer-gpu-4116088a207c)  
21. unslothai/unsloth: Fine-tuning & Reinforcement Learning for LLMs. Train OpenAI gpt-oss, DeepSeek, Qwen, Llama, Gemma, TTS 2x faster with 70% less VRAM. \- GitHub, accessed February 19, 2026, [https://github.com/unslothai/unsloth](https://github.com/unslothai/unsloth)  
22. Qwen3 Fine-tuning now in Unsloth \- 2x faster with 70% less VRAM : r/LocalLLaMA \- Reddit, accessed February 19, 2026, [https://www.reddit.com/r/LocalLLaMA/comments/1kd531l/qwen3\_finetuning\_now\_in\_unsloth\_2x\_faster\_with\_70/](https://www.reddit.com/r/LocalLLaMA/comments/1kd531l/qwen3_finetuning_now_in_unsloth_2x_faster_with_70/)  
23. \~26 tok/sec with Unsloth Qwen3-Coder-Next-Q4\_K\_S on RTX 5090 (Windows/llama.cpp) : r/LocalLLaMA \- Reddit, accessed February 19, 2026, [https://www.reddit.com/r/LocalLLaMA/comments/1qx2teh/26\_toksec\_with\_unsloth\_qwen3codernextq4\_k\_s\_on/](https://www.reddit.com/r/LocalLLaMA/comments/1qx2teh/26_toksec_with_unsloth_qwen3codernextq4_k_s_on/)  
24. Reinforcement Learning (RL) Guide | Unsloth Documentation, accessed February 19, 2026, [https://unsloth.ai/docs/get-started/reinforcement-learning-rl-guide](https://unsloth.ai/docs/get-started/reinforcement-learning-rl-guide)  
25. Deep dive into Group Relative Policy Optimization (GRPO) \- AWS Builder Center, accessed February 19, 2026, [https://builder.aws.com/content/2rJrpj6m2eh591fjMcRZ3ushpB7/deep-dive-into-group-relative-policy-optimization-grpo](https://builder.aws.com/content/2rJrpj6m2eh591fjMcRZ3ushpB7/deep-dive-into-group-relative-policy-optimization-grpo)  
26. Why GRPO is Important and How it Works \- Oxen.ai, accessed February 19, 2026, [https://ghost.oxen.ai/why-grpo-is-important-and-how-it-works/](https://ghost.oxen.ai/why-grpo-is-important-and-how-it-works/)  
27. The Illustrated GRPO: A Detailed and Pedagogical Explanation of Group Relative Policy Optimization (GRPO) Algorithm, accessed February 19, 2026, [https://abderrahmanskiredj.github.io/the-illustrated-grpo/](https://abderrahmanskiredj.github.io/the-illustrated-grpo/)  
28. Group Relative Policy Optimization \- Emergent Mind, accessed February 19, 2026, [https://www.emergentmind.com/topics/group-relative-policy-optimization-grpo](https://www.emergentmind.com/topics/group-relative-policy-optimization-grpo)  
29. Fine-Tuning LLMs with Non-Differentiable Human Feedback \+ RL | by Kanak Raj \- Medium, accessed February 19, 2026, [https://medium.com/tr-labs-ml-engineering-blog/fine-tuning-llms-with-non-differentiable-human-feedback-rl-ec6c33a45928](https://medium.com/tr-labs-ml-engineering-blog/fine-tuning-llms-with-non-differentiable-human-feedback-rl-ec6c33a45928)  
30. Apex Trader Funding Consistency Rule: How It Works \- QuantVPS, accessed February 19, 2026, [https://www.quantvps.com/blog/apex-trader-funding-consistency-rule](https://www.quantvps.com/blog/apex-trader-funding-consistency-rule)  
31. 30% Consistency Rule \- Windfall \- Apex Trader Funding, accessed February 19, 2026, [https://support.apextraderfunding.com/hc/en-us/articles/40463260337819-30-Consistency-Rule-Windfall](https://support.apextraderfunding.com/hc/en-us/articles/40463260337819-30-Consistency-Rule-Windfall)  
32. The Conflict Between LLM Reasoning and Structured Output — Fluid Thinking vs. Rigid Rules | by Kishore Gopalan | Google Cloud \- Medium, accessed February 19, 2026, [https://medium.com/google-cloud/the-conflict-between-llm-reasoning-and-structured-output-fluid-thinking-vs-rigid-rules-e64fb0509d40](https://medium.com/google-cloud/the-conflict-between-llm-reasoning-and-structured-output-fluid-thinking-vs-rigid-rules-e64fb0509d40)  
33. LLM Structured Output in 2026: Stop Parsing JSON with Regex and Do It Right, accessed February 19, 2026, [https://dev.to/pockit\_tools/llm-structured-output-in-2026-stop-parsing-json-with-regex-and-do-it-right-34pk](https://dev.to/pockit_tools/llm-structured-output-in-2026-stop-parsing-json-with-regex-and-do-it-right-34pk)  
34. Structured outputs in vLLM: Guiding AI responses | Red Hat Developer, accessed February 19, 2026, [https://developers.redhat.com/articles/2025/06/03/structured-outputs-vllm-guiding-ai-responses](https://developers.redhat.com/articles/2025/06/03/structured-outputs-vllm-guiding-ai-responses)  
35. Guided Decoding Performance on vLLM and SGLang \- The official SqueezeBits Tech blog, accessed February 19, 2026, [https://blog.squeezebits.com/guided-decoding-performance-vllm-sglang](https://blog.squeezebits.com/guided-decoding-performance-vllm-sglang)  
36. How LLM Structured Decoding works \- Another Dev's Two Cents, accessed February 19, 2026, [https://nishtahir.com/how-llm-structured-decoding-works/](https://nishtahir.com/how-llm-structured-decoding-works/)  
37. Reasoning Outputs \- vLLM, accessed February 19, 2026, [https://docs.vllm.ai/en/latest/features/reasoning\_outputs/](https://docs.vllm.ai/en/latest/features/reasoning_outputs/)  
38. Set Up Automated Trading with NinjaTrader | Hands-Free Strategy Guide, accessed February 19, 2026, [https://ninjatrader.com/futures/blogs/foundations-of-strategy-trading-and-development-part-4-automated-strategy-trading/](https://ninjatrader.com/futures/blogs/foundations-of-strategy-trading-and-development-part-4-automated-strategy-trading/)  
39. Using ZeroMQ within Ninjatrader (assistance needed.) : r/algotrading \- Reddit, accessed February 19, 2026, [https://www.reddit.com/r/algotrading/comments/4isu1f/using\_zeromq\_within\_ninjatrader\_assistance\_needed/](https://www.reddit.com/r/algotrading/comments/4isu1f/using_zeromq_within_ninjatrader_assistance_needed/)  
40. Latency after moving micro-service (using ZeroMQ, C, & Python processes) from 64 bit hardware to 32 bit hardware, but nominal cpu usage \- Stack Overflow, accessed February 19, 2026, [https://stackoverflow.com/questions/60081025/latency-after-moving-micro-service-using-zeromq-c-python-processes-from-64](https://stackoverflow.com/questions/60081025/latency-after-moving-micro-service-using-zeromq-c-python-processes-from-64)  
41. FastAPI how to add ZMQ to eventloop \[duplicate\] \- Stack Overflow, accessed February 19, 2026, [https://stackoverflow.com/questions/61912763/fastapi-how-to-add-zmq-to-eventloop](https://stackoverflow.com/questions/61912763/fastapi-how-to-add-zmq-to-eventloop)  
42. Guided JSON with LLMs: From Raw PDFs to Structured Intelligence | by Doil Kim | Medium, accessed February 19, 2026, [https://medium.com/@kimdoil1211/structured-output-with-guided-json-a-practical-guide-for-llm-developers-6577b2eee98a](https://medium.com/@kimdoil1211/structured-output-with-guided-json-a-practical-guide-for-llm-developers-6577b2eee98a)  
43. Building a Scalable, Low-Latency Real-Time Trading System — Detailed Walkthrough | by Himanshu Jain | Jan, 2026 | Medium, accessed February 19, 2026, [https://medium.com/@himanshu2915j/building-a-scalable-low-latency-real-time-trading-system-detailed-walkthrough-7f7ea0be885c](https://medium.com/@himanshu2915j/building-a-scalable-low-latency-real-time-trading-system-detailed-walkthrough-7f7ea0be885c)  
44. ApexEfficiencyRatio.cs  
45. Unsloth Docs | Unsloth Documentation, accessed February 19, 2026, [https://unsloth.ai/docs](https://unsloth.ai/docs)  
46. Qwen3 \- How to Run & Fine-tune | Unsloth Documentation, accessed February 19, 2026, [https://unsloth.ai/docs/models/qwen3-how-to-run-and-fine-tune](https://unsloth.ai/docs/models/qwen3-how-to-run-and-fine-tune)  
47. Unsloth Notebooks | Unsloth Documentation, accessed February 19, 2026, [https://unsloth.ai/docs/get-started/unsloth-notebooks](https://unsloth.ai/docs/get-started/unsloth-notebooks)  
48. \[2509.09995\] QuantAgent: Price-Driven Multi-Agent LLMs for High-Frequency Trading, accessed February 19, 2026, [https://arxiv.org/abs/2509.09995](https://arxiv.org/abs/2509.09995)  
49. TradingAgents: Multi-Agents LLM Financial Trading Framework, accessed February 19, 2026, [https://tradingagents-ai.github.io/](https://tradingagents-ai.github.io/)

### Reward Functions
Definition of the custom GRPO reward functions for Apex reasoning.

In [ ]:
import re
import json

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks for the presence of <think> and </think> tags."""
    count_text = []
    for completion in completions:
        # Check if the completion contains the required tags
        count = completion.count("<think>") + completion.count("</think>")
        # Reward if both tags are present
        if count == 2:
            count_text.append(0.5)
        else:
            count_text.append(0.0)
    return count_text

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion contains a valid XML structure."""
    pattern = r"<think>.*?</think>\s*\{.*\}"
    matches = [re.search(pattern, r, re.DOTALL) for r in completions]
    return [0.5 if match else 0.0 for match in matches]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that strictly checks if the completion ends with a valid JSON block."""
    pattern = r"^<think>\n.*?\n</think>\n\{.*\}$"
    matches = [re.search(pattern, r, re.DOTALL) for r in completions]
    return [0.5 if match else 0.0 for match in matches]

def apex_efficiency_reward_func(completions, prompts: list[str], **kwargs) -> list[float]:
    """
    Simulates the trade outcome based on the model's JSON decision and calculates the Apex Efficiency Ratio.
    Reward = (Total Net Profit) / (Max Burn Observed)
    """
    rewards = []
    
    # We would typically parse the prompt to get the market context
    # And parse the completion to get the trade action
    
    for completion, prompt in zip(completions, prompts):
        try:
            # Extract JSON block
            json_str = completion[completion.rfind("{"):completion.rfind("}")+1]
            decision = json.loads(json_str)
            
            action = decision.get("execution", {}).get("action", "FLAT")
            
            if action == "FLAT":
                rewards.append(0.1) # Small reward for staying flat in uncertain conditions
                continue

            # Mock Simulation Logic for RLVR
            # In a real scenario, this would check future price data relative to entry
            # For now, we simulate based on "expected burn" in the JSON to reinforce risk awareness
            
            risk_mgmt = decision.get("risk_management", {})
            expected_burn = risk_mgmt.get("expected_burn", 100.0)
            target = risk_mgmt.get("apex_efficiency_target", 2.0)
            
            # Simple heuristic: Reward high efficiency targets
            if expected_burn < 50.0:
                 rewards.append(2.0)
            elif expected_burn > 200.0:
                 rewards.append(-1.0) # Penalty for high burn risk
            else:
                 rewards.append(0.5)

        except Exception:
            rewards.append(-1.0) # Penalty for invalid JSON/Execution
            
    return rewards

def regime_consistency_reward_func(completions, prompts: list[str], **kwargs) -> list[float]:
    """
    Rewards the model if the 'market_regime' classification aligns with the prompted indicators.
    """
    rewards = []
    for completion, prompt in zip(completions, prompts):
        try:
            # Extract JSON
            json_str = completion[completion.rfind("{"):completion.rfind("}")+1]
            data = json.loads(json_str)
            
            regime = data.get("market_regime", {})
            tf_60min_regime = regime.get("tf_60min", "")
            
            # Check prompt for "Bullish" or "Bearish" keyword in 60m section
            # Ideally strict parsing, but keyword search works for RLVR speed
            if "Structural Trend: Bullish" in prompt and "Bullish" in tf_60min_regime:
                rewards.append(1.0)
            elif "Structural Trend: Bearish" in prompt and "Bearish" in tf_60min_regime:
                 rewards.append(1.0)
            else:
                 rewards.append(0.0)
                 
        except Exception:
            rewards.append(0.0)
            
    return rewards


### GRPO Training Script
Based on `src/training/train_grpo.py`, modified to run in Colab directly.

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
from unsloth import is_bfloat16_supported
from trl import GRPOTrainer, GRPOConfig
import torch
from datasets import load_dataset
import os

# 1. Patch GRPO for memory efficiency
PatchFastRL("GRPO", FastLanguageModel)

max_seq_length = 2048 
lora_rank = 32 

# 2. Load Model & Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit", 
    max_seq_length = max_seq_length,
    load_in_4bit = True, 
    fast_inference = False, # Disable vLLM fast inference (Windows incompatibility) / stable on Colab
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, 
)

# Force disable native thinking in both model and tokenizer
if hasattr(model.config, "enable_thinking"):
    model.config.enable_thinking = False
if hasattr(tokenizer, "enable_thinking"):
    tokenizer.enable_thinking = False
if hasattr(model, "generation_config") and hasattr(model.generation_config, "enable_thinking"):
    model.generation_config.enable_thinking = False

# Left padding is MANDATORY for GRPO generation
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3. Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# 4. Load Dataset
# Ensure that you upload your data/train.json to Colab before execution!
try:
    dataset = load_dataset("json", data_files="data/train.json", split="train") 

    def format_prompt(example):
        messages = [
            {
                "role": "system", 
                "content": "You are a helpful assistant. You must first think about the answer within <think> tags, then provide the final answer within <answer> tags."
            },
            {
                "role": "user", 
                "content": example["prompt"]
            }
        ]
        formatted = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True 
        )
        return {"prompt": formatted}

    dataset = dataset.map(format_prompt)

    # 5. Configure GRPO Trainer
    training_args = GRPOConfig(
        use_vllm = False, 
        learning_rate = 5e-6,
        adam_beta1 = 0.9,
        adam_beta2 = 0.99,
        weight_decay = 0.1,
        warmup_ratio = 0.1,
        lr_scheduler_type = "cosine",
        optim = "paged_adamw_8bit",
        logging_steps = 1,
        bf16 = is_bfloat16_supported(),
        fp16 = not is_bfloat16_supported(),
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, 
        num_generations = 6,            
        max_prompt_length = 768,        
        max_completion_length = 512,    
        max_steps = 250,
        save_steps = 250,
        max_grad_norm = 0.1,
        report_to = "none", 
        output_dir = "outputs/qwen3-grpo",
    )

    trainer = GRPOTrainer(
        model = model,
        processing_class = tokenizer,
        reward_funcs = [
            xmlcount_reward_func,
            soft_format_reward_func,
            strict_format_reward_func,
            apex_efficiency_reward_func,
            regime_consistency_reward_func,
        ],
        args = training_args,
        train_dataset = dataset,
    )

    # 6. Train
    print("Starting GRPO Training...")
    trainer.train()

    # 7. Save Model
    print("Saving model and LoRA adapters...")
    model.save_lora("outputs/qwen3-grpo/lora_adapters")
except Exception as e:
    print(f"Error loading dataset or starting training (make sure data/train.json is uploaded): {e}")
